# Azure AI-agenter med stöd för Model Context Protocol (MCP) - Python

Den här notebooken visar hur man använder Azure AI-agenter med verktyg för Model Context Protocol (MCP) i Python. Den visar hur man skapar en intelligent agent som kan utnyttja externa MCP-servrar (som Microsoft Learn) för förbättrade funktioner med nyckellös autentisering.


## Installera nödvändiga Python-paket

Först behöver vi installera de nödvändiga Python-paketen:
- **azure-ai-projects**: Kärn-SDK för Azure AI Projects
- **azure-ai-agents**: Azure AI Agents SDK för att skapa och hantera agenter
- **azure-identity**: Ger nyckellös autentisering med DefaultAzureCredential
- **mcp**: Implementering av Model Context Protocol för Python


## Fördelar med nyckellös autentisering

Den här notebooken demonstrerar **nyckellös autentisering**, som erbjuder flera fördelar:
- ✅ **Inga API-nycklar att hantera** - Använder Azure-baserad identitetsautentisering
- ✅ **Förbättrad säkerhet** - Inga hemligheter lagras i kod eller konfigurationsfiler
- ✅ **Automatisk rotation av autentiseringsuppgifter** - Azure hanterar livscykelhantering av autentiseringsuppgifter
- ✅ **Rollbaserad åtkomstkontroll** - Använder Azure RBAC för detaljerad behörighetsstyrning
- ✅ **Stöd för flera miljöer** - Fungerar smidigt både i utvecklings- och produktionsmiljöer

`DefaultAzureCredential` väljer automatiskt den bästa tillgängliga autentiseringskällan:
1. **Managed Identity** (när den körs i Azure)
2. **Azure CLI**-autentiseringsuppgifter (under lokal utveckling)
3. **Visual Studio**-autentiseringsuppgifter
4. **Miljövariabler** (om de är konfigurerade)
5. **Interaktiv webbläsarautentisering** (som reserv)


## Konfigurationsinställningar för nyckellös autentisering

**Förutsättningar för nyckellös autentisering:**

### För lokal utveckling:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### För Azure-miljöer:
- Aktivera **Systemtilldelad Hanterad Identitet** på din Azure-resurs
- Tilldela lämpliga **RBAC-roller** till den hanterade identiteten:
  - `Cognitive Services OpenAI User` för åtkomst till Azure OpenAI
  - `AI Developer` för åtkomst till Azure AI-projekt

### Miljövariabler (Valfritt):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Inga API-nycklar eller anslutningssträngar behövs!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Importera Nödvändiga Bibliotek

Importera de nödvändiga Python-modulerna:
- **os, time**: Standardbibliotek i Python för miljövariabler och fördröjningar
- **AIProjectClient**: Huvudklient för Azure AI-projekt
- **DefaultAzureCredential**: Nyckellös autentisering för Azure-tjänster
- **MCP-relaterade klasser**: För att skapa och hantera MCP-verktyg samt hantera godkännanden


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Konfigurera MCP-serverinställningar

Ställ in MCP-serverns konfiguration med hjälp av miljövariabler med reservstandardvärden:
- **MCP_SERVER_URL**: URL till MCP-servern (standard är Microsoft Learn API)
- **MCP_SERVER_LABEL**: En etikett för att identifiera MCP-servern (standard är "mslearn")

Denna metod möjliggör flexibel konfiguration i olika miljöer.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Skapa Azure AI Project-klient (Nyckelfri autentisering)

Initiera Azure AI Project-klienten med **nyckelfri autentisering**:
- **endpoint**: URL för Azure AI Foundry-projektets slutpunkt
- **credential**: Använder `DefaultAzureCredential()` för säker, nyckelfri autentisering
- **Inga API-nycklar krävs**: Upptäcker och använder automatiskt den bästa tillgängliga autentiseringsmetoden

**Autentiseringsflöde:**
1. Kontrollerar för Managed Identity (i Azure-miljöer)
2. Går över till Azure CLI-autentisering (för lokal utveckling)
3. Använder andra tillgängliga autentiseringskällor vid behov

Denna metod eliminerar behovet av att hantera API-nycklar eller anslutningssträngar i din kod.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Skapa MCP-verktygsdefinition

Skapa ett MCP-verktyg som ansluter till Microsoft Learn MCP-servern:
- **server_label**: Identifierare för MCP-servern
- **server_url**: URL-slutpunkt för MCP-servern
- **allowed_tools**: Valfri lista för att begränsa vilka verktyg som kan användas (tom lista tillåter alla verktyg)

Detta verktyg gör det möjligt för agenten att få tillgång till Microsoft Learn-dokumentation och resurser.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Skapa agent och genomför konversation (Nyckellös arbetsflöde)

Denna omfattande sektion visar det kompletta **nyckellösa agentarbetsflödet**:

1. **Skapa AI-agent**: Ställ in en agent med GPT-4.1 nano-modell och MCP-verktyg
2. **Skapa tråd**: Etablera en konversationstråd för kommunikation
3. **Skicka meddelande**: Fråga agenten om skillnaderna mellan Azure OpenAI och OpenAI
4. **Hantera verktygsgodkännanden**: Godkänn automatiskt MCP-verktygsanrop när det behövs
5. **Övervaka utförande**: Följ agentens framsteg och hantera eventuella nödvändiga åtgärder
6. **Visa resultat**: Visa konversationen och detaljer om verktygsanvändning

**Nyckellösa funktioner:**
- ✅ **Inga hårdkodade hemligheter** - All autentisering hanteras av Azure-identitet
- ✅ **Säkert som standard** - Använder rollbaserad åtkomstkontroll
- ✅ **Förenklad distribution** - Ingen hantering av autentiseringsuppgifter krävs
- ✅ **Revisionsvänligt** - All åtkomst spåras via Azure-identitet

Agenten kommer att använda MCP-verktyg för att få åtkomst till Microsoft Learn-resurser med full säkerhet och utan hantering av API-nycklar.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör det noteras att automatiserade översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som kan uppstå vid användning av denna översättning.
